In [97]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils import to_categorical

from Bio import SeqIO
from Bio.Data import IUPACData 
import csv
import numpy as np
import tensorflow as tf

#data_path = 'features_CENH3_DMR6_LUCA-CHLRE00002_orthologues.csv'
data_path = 'features_oma-seqs-viridiplantae_test-4-5-6-7.csv'

In [81]:
def protein2integer(in_seq):
    
    ## define universe of possible input values
    all_protein_letters = list(IUPACData.extended_protein_letters)
    #print(all_protein_letters)
    ## define a mapping of chars to integers 
    ## i+1 beacuse we want to start from integer 1 instead of 0. 0 will be used for padding
    char_to_int = dict((c, i+1) for i, c in enumerate(all_protein_letters))
    int_to_char = dict((i+1, c) for i, c in enumerate(all_protein_letters))
    ## integer encode input data
    integer_encoded = [char_to_int[char] for char in in_seq.upper()]
    
    
    #return(integer_encoded,len(all_protein_letters))
    return(integer_encoded)
    

In [82]:
def make_dataset(in_file):
    with open(in_file, 'r') as f:
        reader = csv.reader(f, delimiter="\t")
        # get all the rows as a list
        d_set = list(reader)
        # transform data into numpy array
        d_set = np.array(d_set).astype(str)
        
    integer_encoded_proteins = np.array([protein2integer(seq) for seq in d_set[:,1]])
    
    G = d_set[:, 0]
    X = integer_encoded_proteins
    Y = d_set[:, 2].astype(int)
                         
    return(d_set,G,X,Y)

In [83]:
def make_train_test_set(G,X,Y):
    indices = np.random.permutation(X.shape[0])
    train_size = int(indices.size*0.90)
    train_idx, test_idx = indices[:train_size], indices[train_size:]
    #print(len(train_idx),len(test_idx))
    
    X_train, X_test = X[train_idx,], X[test_idx,]
    #print(X_train.shape,X_test.shape)
    
    y_train, y_test = Y[train_idx,], Y[test_idx,]
    
    #print(X[train_idx[0],])
    #print(Y[train_idx[0],])

    #print(X_train[0,])
    #print(y_train[0,])
    
    return(X_train,y_train,X_test,y_test)

In [106]:
def model1(X_train_new, y_train,X_test_new, y_test,in_batch_size=100,in_epochs=10): # RNN: Recurrent Neural Networks
    # https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
    # create the model
    n_classes = int(np.amax(np.concatenate((y_train,y_test),axis=0))+1)
    embedding_vecor_length = 32
    model = Sequential()
    model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(100))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])
    print(model.summary())
    
    
    #model.fit(X_train_new, y_train, epochs=in_epochs, batch_size=in_batch_size) #epochs=3, batch_size=64)
    ## Final evaluation of the model
    #scores = model.evaluate(X_test_new, y_test, verbose=0)
    #print("Accuracy: %.2f%%" % (scores[1]*100))
    
    # Convert labels to categorical one-hot encoding
    y_train_one_hot_labels = to_categorical(y_train, num_classes=n_classes)
    model.fit(X_train_new, y_train_one_hot_labels, epochs=in_epochs, batch_size=in_batch_size)
    
    y_test_one_hot_labels = to_categorical(y_test, num_classes=n_classes)
    scores = model.evaluate(X_test_new, y_test_one_hot_labels, verbose=0)
    
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))
    
    return()


In [101]:
def model2(X_train_new, y_train,X_test_new, y_test,in_batch_size=100,in_epochs=10): # RNN: Recurrent Neural Networks
    # Initializing the Sequential model from KERAS.
    model = Sequential()

    max_review_length = len(max(X, key=len))
    n_classes = int(np.amax(np.concatenate((y_train,y_test),axis=0))+1)

    # Creating a 16 neuron hidden layer with Linear Rectified activation function.
    #model.add(Dense(16, input_dim=1, init='uniform', activation='relu'))
    model.add(Dense(16, input_dim=max_review_length, kernel_initializer='uniform', activation='relu'))

    # Creating a 8 neuron hidden layer.
    model.add(Dense(8, kernel_initializer='uniform', activation='relu'))

    # Adding a output layer.
    model.add(Dense(n_classes, kernel_initializer='uniform', activation='softmax'))
    
    # Compiling the model
    model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])

    print(model.summary())

    # Fitting the model
    #model.fit(X, Y, nb_epoch=150, batch_size=10)
    #scores = model.evaluate(X, Y)
    
    #model.fit(X_train_new, y_train, epochs=in_epochs, batch_size=in_batch_size)
    #scores = model.evaluate(X_test_new, y_test, verbose=0)
    
    # Convert labels to categorical one-hot encoding
    y_train_one_hot_labels = to_categorical(y_train, num_classes=n_classes)
    model.fit(X_train_new, y_train_one_hot_labels, epochs=in_epochs, batch_size=in_batch_size)
    
    y_test_one_hot_labels = to_categorical(y_test, num_classes=n_classes)
    scores = model.evaluate(X_test_new, y_test_one_hot_labels, verbose=0)
    

    print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))
    
    return()

In [102]:
def one_hot_matrix(labels,C):
    
    C = tf.constant(C,name="C")
    one_hot_matrix = tf.one_hot(labels,C,axis=1)
    sess = tf.Session()
    one_hot = sess.run(one_hot_matrix)
    sess.close()
    
    return one_hot

In [103]:
def model3(X_train_new, y_train,X_test_new, y_test, batch_size =100, hm_epochs =100): # CNN: Convolutional Neural Networks
    # Number of nodes in each NN hidden layer
    n_nodes_hl1 = 1500
    n_nodes_hl2 = 1500
    n_nodes_hl3 = 1500

    # Number of orthology clusters
    #n_classes = len(np.unique(np.concatenate((y_train,y_test),axis=0)))     #2 or 3 or ...
    
    #y_all = np.concatenate((y_train,y_test),axis=0)
    #y_min = np.amin(y_all)
    #n_classes = np.amax(y_all-y_min)+1
    n_classes = int(np.amax(np.concatenate((y_train,y_test),axis=0))+1)
    
    train_y = one_hot_matrix(y_train,n_classes)
    test_y = one_hot_matrix(y_test,n_classes)

    # Batch size and Epoch size for training the NN
    #batch_size = 100   #100
    #hm_epochs = 100    #1000

    # Initializing X and Y
    x = tf.placeholder('float')
    y = tf.placeholder('float')

    # Initializing NN layers
    hidden_1_layer = {'f_fum':n_nodes_hl1,
                  'weight':tf.Variable(tf.random_normal([len(X_train_new[0]), n_nodes_hl1])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'f_fum':n_nodes_hl2,
                  'weight':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'f_fum':n_nodes_hl3,
                  'weight':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'f_fum':None,
                'weight':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                'bias':tf.Variable(tf.random_normal([n_classes])),}


    
    
    l1 = tf.add(tf.matmul(x,hidden_1_layer['weight']), hidden_1_layer['bias'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weight']), hidden_2_layer['bias'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weight']), hidden_3_layer['bias'])
    l3 = tf.nn.relu(l3)

    prediction = tf.matmul(l3,output_layer['weight']) + output_layer['bias']

        
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=y) )
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        #try:
        #    epoch = int(open(tf_log,'r').read().split('\n')[-2])+1
        #    print('STARTING:',epoch)
        #except:
        #    epoch = 1
        epoch = 1

        while epoch <= hm_epochs:
            epoch_loss = 1
            
            i=0
            while i < len(X_train_new):
                start = i
                end = i+batch_size
                batch_x = np.array(X_train_new[start:end])
                batch_y = np.array(train_y[start:end])

                _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,y: batch_y})
                epoch_loss += c
                i+=batch_size
                
            
            print('Epoch ',epoch,' out of ',hm_epochs,'- loss:',epoch_loss)
 
            
            #with open(tf_log,'a') as f:
            #    f.write(str(epoch)+'\n') 
            epoch +=1
            
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

        #print("\nModel saved in path: %s " % my_model_save_path)
        print('\nAccuracy:',accuracy.eval({x:X_test_new, y:test_y}) * 100)
    return()

In [88]:
def test_1(d_set):
    print(d_set[:,0])
    print(d_set[0:2,1])
    print(d_set[:,2].astype(int))
    print(d_set.shape)
    print(d_set[:,1].shape)
    return()

In [89]:
def test_2(d_set):
    integer_encoded_proteins = np.array([protein2integer(seq) for seq in d_set[:,1]])
    print(len(integer_encoded_proteins))
    print(integer_encoded_proteins[0])
    #np.array(integer_encoded_proteins).shape
    print(integer_encoded_proteins.shape)
    #protein2integer(dataset[:,1])
    return()

In [90]:
def test_3(G,X,Y):
    print(G.shape)
    print(X.shape)
    print(Y.shape)

    print(G[0:3,])
    print(X[0:3,])
    print(Y[0:3,])
    
    print('Maximum review length: {}'.format(len(max(X, key=len))))
    print('Minimum review length: {}'.format(len(min(X, key=len))))
    
    return()

In [91]:
def test_4(X_train_new,X_train):
    print(X_train_new.shape)
    print(X_train_new[0,:])
    print(X_train.shape)
    print(X_train[0,])
    return()


In [92]:
dataset, G, X, Y = make_dataset(data_path)
X_train,y_train,X_test,y_test = make_train_test_set(G,X,Y)

#print("============ Test 1 =======================")
#test_1(dataset)
#print("============ Test 2 =======================")
#test_2(dataset)
#print("============ Test 3 =======================")
#test_3(G,X,Y)



In [93]:
top_words = len(list(IUPACData.extended_protein_letters)) # = 26
max_review_length = len(max(X, key=len)) # = 500
# truncate and pad input sequences
X_train_new = sequence.pad_sequences(X_train, maxlen=max_review_length, padding='post', truncating='post')
X_test_new = sequence.pad_sequences(X_test, maxlen=max_review_length, padding='post', truncating='post')
  
#print("============ Test 4 =======================")
#test_4(X_train_new,X_train)


In [ ]:
model1(X_train_new, y_train,X_test_new, y_test,50,10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 5068, 32)          832       
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 5068, 32)          3104      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 2534, 32)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_28 (Dense)             (None, 16141)             1630241   
Total params: 1,687,377
Trainable params: 1,687,377
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
 5750/11662 [=============>................] - ETA: 3:26 - loss: 9.2058 - acc: 0.0000e+00

In [100]:
model2(X_train_new, y_train,X_test_new, y_test,50,100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 16)                81104     
_________________________________________________________________
dense_24 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_25 (Dense)             (None, 16141)             145269    
Total params: 226,509
Trainable params: 226,509
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
11662/11662 [==============================] - 7s 587us/step - loss: 9.0372 - acc: 3.4299e-04
Epoch 2/100
11662/11662 [==============================] - 6s 543us/step - loss: 7.7281 - acc: 0.0012
Epoch 3/100
11662/11662 [==============================] - 7s 576us/step - loss: 7.1512 - acc: 0.0021
Epoch 4/100
11662/11662 [==============================] - 7s 607us/step - loss: 6.77

11662/11662 [==============================] - 7s 563us/step - loss: 4.6990 - acc: 0.0969
Epoch 74/100
11662/11662 [==============================] - 6s 539us/step - loss: 4.6877 - acc: 0.1001
Epoch 75/100
11662/11662 [==============================] - 6s 525us/step - loss: 4.6866 - acc: 0.0989
Epoch 76/100
11662/11662 [==============================] - 7s 578us/step - loss: 4.6750 - acc: 0.1005
Epoch 77/100
11662/11662 [==============================] - 7s 592us/step - loss: 4.6710 - acc: 0.1013
Epoch 78/100
11662/11662 [==============================] - 6s 531us/step - loss: 4.6601 - acc: 0.1006
Epoch 79/100
11662/11662 [==============================] - 7s 561us/step - loss: 4.6590 - acc: 0.1003
Epoch 80/100
11662/11662 [==============================] - 6s 544us/step - loss: 4.6559 - acc: 0.1001
Epoch 81/100
11662/11662 [==============================] - 6s 549us/step - loss: 4.6495 - acc: 0.1012
Epoch 82/100
11662/11662 [==============================] - 6s 544us/step - loss: 4.62

()

In [76]:
model3(X_train_new,y_train,X_test_new,y_test,100,50)

('Epoch ', 1, ' out of ', 50, '- loss:', 787625906.25)
('Epoch ', 2, ' out of ', 50, '- loss:', 146818675.71875)
('Epoch ', 3, ' out of ', 50, '- loss:', 10033231.520507812)
('Epoch ', 4, ' out of ', 50, '- loss:', 1455508.5444335938)
('Epoch ', 5, ' out of ', 50, '- loss:', 560215.0004272461)
('Epoch ', 6, ' out of ', 50, '- loss:', 316796.73611831665)
('Epoch ', 7, ' out of ', 50, '- loss:', 191133.06982421875)
('Epoch ', 8, ' out of ', 50, '- loss:', 133056.8287382126)
('Epoch ', 9, ' out of ', 50, '- loss:', 65381.50121688843)
('Epoch ', 10, ' out of ', 50, '- loss:', 43988.85781955719)
('Epoch ', 11, ' out of ', 50, '- loss:', 30168.841267585754)
('Epoch ', 12, ' out of ', 50, '- loss:', 22453.99934387207)
('Epoch ', 13, ' out of ', 50, '- loss:', 12963.844420433044)
('Epoch ', 14, ' out of ', 50, '- loss:', 8075.836024284363)
('Epoch ', 15, ' out of ', 50, '- loss:', 4806.231784820557)
('Epoch ', 16, ' out of ', 50, '- loss:', 6616.773619651794)
('Epoch ', 17, ' out of ', 50, '- 

()